
Julia is not fully thread-safe yet.

# Julia Tasks (aka Coroutines) 

## Channels

Channels can be quite useful to pass data between running tasks, particularly those involving I/O operations.



In [15]:
# Given Channels c1 and c2,
c1 = Channel(32) # can hold a maximum of 32 objects of any type.
c2 = Channel(32)

# and a function `slow_double` which reads items from from c1, 
# doubles the item read, wait 1 second
# and writes a result to c2,
function slow_double()
    while true
        data = take!(c1)
        sleep(1)
        result = 2 * data   # process data
        put!(c2, result)    # write out result
    end
end

n = 8
# we can schedule `n` instances of `foo` to be active concurrently.
for _ in 1:n
    @async slow_double()
end

In [1]:
using Distributed
workers()

1-element Array{Int64,1}:
 1

In [2]:
addprocs(3)

3-element Array{Int64,1}:
 2
 3
 4

In [3]:
workers() # proc 1 is now the master and is not a worker anymore

3-element Array{Int64,1}:
 2
 3
 4

In [6]:
psize = 8
@distributed for prank=1:psize
    println(myid()); # return worker id
end;

      From worker 2:	2
      From worker 2:	2
      From worker 2:	2
      From worker 3:	3
      From worker 4:	4
      From worker 4:	4
      From worker 3:	3
      From worker 3:	3


In [8]:
using SharedArrays
psize = 8
a = SharedArray{Int}(1,psize) # Tableau partagé
@distributed for prank=1:psize
    a[i] = myid()
end

  likely near In[8]:3


Task (runnable) @0x000000011b2c5450

In [10]:
collect(a)

1×8 Array{Int64,2}:
 0  0  0  0  0  0  0  0